In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.linear_model import LinearRegression
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import BaggingRegressor
from sklearn.metrics import root_mean_squared_error,r2_score

## Carregar os dados

In [2]:
df_taxi_pricing = pd.read_csv('./taxi_trip_pricing.csv')

In [3]:
df_taxi_pricing.head()

,Trip_Distance_km,Time_of_Day,Day_of_Week,Passenger_Count,Traffic_Conditions,Weather,Base_Fare,Per_Km_Rate,Per_Minute_Rate,Trip_Duration_Minutes,Trip_Price
0,19.35,Morning,Weekday,3.0,Low,Clear,3.56,0.80,0.32,53.82,36.2624
1,47.59,Afternoon,Weekday,1.0,High,Clear,NaN,0.62,0.43,40.57,NaN
2,36.87,Evening,Weekend,1.0,High,Clear,2.70,1.21,0.15,37.27,52.9032
3,30.33,Evening,Weekday,4.0,Low,NaN,3.48,0.51,0.15,116.81,36.4698
4,NaN,Evening,Weekday,3.0,High,Clear,2.93,0.63,0.32,22.64,15.6180


In [4]:
df_taxi_pricing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Trip_Distance_km       950 non-null    float64
 1   Time_of_Day            950 non-null    object 
 2   Day_of_Week            950 non-null    object 
 3   Passenger_Count        950 non-null    float64
 4   Traffic_Conditions     950 non-null    object 
 5   Weather                950 non-null    object 
 6   Base_Fare              950 non-null    float64
 7   Per_Km_Rate            950 non-null    float64
 8   Per_Minute_Rate        950 non-null    float64
 9   Trip_Duration_Minutes  950 non-null    float64
 10  Trip_Price             951 non-null    float64
dtypes: float64(7), object(4)
memory usage: 86.1+ KB


## EDA

In [5]:
df_taxi_pricing.describe()

,Trip_Distance_km,Passenger_Count,Base_Fare,Per_Km_Rate,Per_Minute_Rate,Trip_Duration_Minutes,Trip_Price
count,950.000000,950.000000,950.000000,950.000000,950.000000,950.000000,951.000000
mean,27.070547,2.476842,3.502989,1.233316,0.292916,62.118116,56.874773
std,19.905300,1.102249,0.870162,0.429816,0.115592,32.154406,40.469791
min,1.230000,1.000000,2.010000,0.500000,0.100000,5.010000,6.126900
25%,12.632500,1.250000,2.730000,0.860000,0.190000,35.882500,33.742650
50%,25.830000,2.000000,3.520000,1.220000,0.290000,61.860000,50.074500
75%,38.405000,3.000000,4.260000,1.610000,0.390000,89.055000,69.099350
max,146.067047,4.000000,5.000000,2.000000,0.500000,119.840000,332.043689


In [6]:
contagem_time_of_day = df_taxi_pricing.value_counts('Time_of_Day')

px.bar(contagem_time_of_day/len(df_taxi_pricing) * 100, color=contagem_time_of_day.index)

In [7]:
contagem_day_of_week = df_taxi_pricing.value_counts('Day_of_Week')

px.bar(contagem_day_of_week/len(df_taxi_pricing) * 100, color=contagem_day_of_week.index)

In [8]:
contagem_weather = df_taxi_pricing.value_counts('Weather')

px.bar(contagem_weather/len(df_taxi_pricing) * 100, color=contagem_weather.index)

In [9]:

contagem_traffic_conditions = df_taxi_pricing.value_counts('Traffic_Conditions')

px.bar(contagem_traffic_conditions/len(df_taxi_pricing) * 100, color=contagem_traffic_conditions.index)

In [10]:
px.histogram(df_taxi_pricing,x='Per_Km_Rate')

In [11]:
px.histogram(df_taxi_pricing,x='Trip_Distance_km')

## Feature Engenieer

In [12]:
# Verificando se há valores nulos

for col in df_taxi_pricing.select_dtypes(include=['number']).columns:
    contagem_nulos = df_taxi_pricing[col].isnull().sum()
    print(f"Coluna: {col}, Valores nulos: {contagem_nulos} - {contagem_nulos/len(df_taxi_pricing) * 100} %")

Coluna: Trip_Distance_km, Valores nulos: 50 - 5.0 %
Coluna: Passenger_Count, Valores nulos: 50 - 5.0 %
Coluna: Base_Fare, Valores nulos: 50 - 5.0 %
Coluna: Per_Km_Rate, Valores nulos: 50 - 5.0 %
Coluna: Per_Minute_Rate, Valores nulos: 50 - 5.0 %
Coluna: Trip_Duration_Minutes, Valores nulos: 50 - 5.0 %
Coluna: Trip_Price, Valores nulos: 49 - 4.9 %


In [13]:
for col in df_taxi_pricing.select_dtypes(include=['number']).columns:
    df_taxi_pricing[col] = df_taxi_pricing[col].fillna(np.mean(df_taxi_pricing[col]))

In [14]:
df_taxi_pricing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Trip_Distance_km       1000 non-null   float64
 1   Time_of_Day            950 non-null    object 
 2   Day_of_Week            950 non-null    object 
 3   Passenger_Count        1000 non-null   float64
 4   Traffic_Conditions     950 non-null    object 
 5   Weather                950 non-null    object 
 6   Base_Fare              1000 non-null   float64
 7   Per_Km_Rate            1000 non-null   float64
 8   Per_Minute_Rate        1000 non-null   float64
 9   Trip_Duration_Minutes  1000 non-null   float64
 10  Trip_Price             1000 non-null   float64
dtypes: float64(7), object(4)
memory usage: 86.1+ KB


In [15]:
# Verificando se há valores nulos

for col in df_taxi_pricing.select_dtypes(include=['object']).columns:
    contagem_nulos = df_taxi_pricing[col].isnull().sum()
    print(f"Coluna: {col}, Valores nulos: {contagem_nulos} - {contagem_nulos/len(df_taxi_pricing) * 100} %")

Coluna: Time_of_Day, Valores nulos: 50 - 5.0 %
Coluna: Day_of_Week, Valores nulos: 50 - 5.0 %
Coluna: Traffic_Conditions, Valores nulos: 50 - 5.0 %
Coluna: Weather, Valores nulos: 50 - 5.0 %


In [16]:
for col in df_taxi_pricing.select_dtypes(include=['object']).columns:
    df_taxi_pricing[col] = df_taxi_pricing[col].fillna(df_taxi_pricing[col].mode()[0])

In [17]:
df_taxi_pricing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Trip_Distance_km       1000 non-null   float64
 1   Time_of_Day            1000 non-null   object 
 2   Day_of_Week            1000 non-null   object 
 3   Passenger_Count        1000 non-null   float64
 4   Traffic_Conditions     1000 non-null   object 
 5   Weather                1000 non-null   object 
 6   Base_Fare              1000 non-null   float64
 7   Per_Km_Rate            1000 non-null   float64
 8   Per_Minute_Rate        1000 non-null   float64
 9   Trip_Duration_Minutes  1000 non-null   float64
 10  Trip_Price             1000 non-null   float64
dtypes: float64(7), object(4)
memory usage: 86.1+ KB


## Preparação dos dados

In [18]:
X = df_taxi_pricing.drop(columns=['Trip_Price'], axis=1)
y = df_taxi_pricing['Trip_Price']

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)

In [20]:
features_numericas = X.select_dtypes(include=['float64']).columns
features_categoricas = X.select_dtypes(include=['object']).columns

preprocessor = ColumnTransformer(
    transformers=[
        ('num',StandardScaler(),features_numericas),
        ('cat', OneHotEncoder(handle_unknown='ignore'), features_categoricas)
    ],
    remainder='passthrough'
)

In [21]:
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

## Treinamento do modelo

In [22]:
# Mostrar as dimensões dos conjuntos
print(f'Dados de treinamento: {X_train.shape}')
print(f'Dados de teste: {X_test.shape}')

Dados de treinamento: (800, 18)
Dados de teste: (200, 18)


In [23]:
model_bagging = BaggingRegressor(
    estimator=LinearRegression(),
    n_estimators=30,
    random_state=51
)

In [24]:
model_bagging.fit(X_train,y_train)

,"estimator estimator: object, default=NoneThe base estimator to fit on random subsets of the dataset.If None, then the base estimator is a:class:`~sklearn.tree.DecisionTreeRegressor`... versionadded:: 1.2 `base_estimator` was renamed to `estimator`.",LinearRegression()
,"n_estimators n_estimators: int, default=10The number of base estimators in the ensemble.",30
,"max_samples max_samples: int or float, default=NoneThe number of samples to draw from X to train each base estimator (withreplacement by default, see `bootstrap` for more details).- If None, then draw `X.shape[0]` samples irrespective of `sample_weight`.- If int, then draw `max_samples` samples.- If float, then draw `max_samples * X.shape[0]` unweighted samples or `max_samples * sample_weight.sum()` weighted samples.",None
,"max_features max_features: int or float, default=1.0The number of features to draw from X to train each base estimator (without replacement by default, see `bootstrap_features` for moredetails).- If int, then draw `max_features` features.- If float, then draw `max(1, int(max_features * n_features_in_))` features.",1.0
,"bootstrap bootstrap: bool, default=TrueWhether samples are drawn with replacement. If False, sampling withoutreplacement is performed. If fitting with `sample_weight`, it isstrongly recommended to choose True, as only drawing with replacementwill ensure the expected frequency semantics of `sample_weight`.",True
,"bootstrap_features bootstrap_features: bool, default=FalseWhether features are drawn with replacement.",False
,"oob_score oob_score: bool, default=FalseWhether to use out-of-bag samples to estimatethe generalization error. Only available if bootstrap=True.",False
,"warm_start warm_start: bool, default=FalseWhen set to True, reuse the solution of the previous call to fitand add more estimators to the ensemble, otherwise, just fita whole new ensemble. See :term:`the Glossary `.",False
,"n_jobs n_jobs: int, default=NoneThe number of jobs to run in parallel for both :meth:`fit` and:meth:`predict`. ``None`` means 1 unless in a:obj:`joblib.parallel_backend` context. ``-1`` means using allprocessors. See :term:`Glossary ` for more details.",None
,"random_state random_state: int, RandomState instance or None, default=NoneControls the random resampling of the original dataset(sample wise and feature wise).If the base estimator accepts a `random_state` attribute, a differentseed is generated for each instance in the ensemble.Pass an int for reproducible output across multiple function calls.See :term:`Glossary `.",51
,"verbose verbose: int, default=0Controls the verbosity when fitting and predicting.",0


In [25]:
y_pred = model_bagging.predict(X_test)

In [26]:
y_pred

array([ 7.51635539e+01,  8.90325626e+01,  6.35735855e+01,  7.31229653e+01,
        1.21868576e+02,  7.63334958e+01,  2.52545680e+01,  5.55662703e+01,
        2.94512941e+01,  7.74857035e+01,  7.46063338e+01,  1.02536998e+02,
        7.44782941e+01,  2.51256042e+01, -3.03623156e-01,  1.23891810e+00,
        7.10041412e+01,  1.53427571e+01,  1.93597380e+01,  3.72466718e+01,
        7.94928424e+01,  3.87402679e+01, -2.40200780e+01,  3.49601260e+01,
        4.48068591e+01,  9.22397573e+01,  6.75546340e+01,  4.30936188e+01,
        3.46734477e+01,  9.47487845e+01,  8.43428824e+01,  8.72474506e+01,
        1.29893306e+01,  1.06130544e+02,  3.91531634e+01,  9.58361254e+01,
        2.20618961e+01,  5.89503565e+01,  2.58571835e+01,  7.67207159e+01,
        5.24037434e+01,  3.80141075e+01, -8.69696392e-02,  3.24037526e+01,
        1.14859553e+01,  5.98786175e+01,  5.33379653e+01,  4.41470444e+01,
        4.29559429e+01,  4.97805028e+01,  6.38697848e+01,  6.97522273e+00,
        8.39714731e+01,  

## Avaliando o modelo

In [27]:
rmse = root_mean_squared_error(y_test,y_pred)
r2 = r2_score(y_test,y_pred)

In [28]:
print(f'RMSE : {rmse}')
print(f'R2 Score : {r2}')

RMSE : 14.015622505422614
R2 Score : 0.7634318626895442


In [29]:
importances = np.mean([np.abs(estimator.coef_) for estimator in model_bagging.estimators_], axis=0)
importances_norm = importances/np.sum(importances)

importances_norm

array([0.47526436, 0.00669021, 0.00916142, 0.13897897, 0.07486256,
       0.11967846, 0.013879  , 0.0143485 , 0.02007978, 0.0198133 ,
       0.00670143, 0.00670143, 0.01423281, 0.00814433, 0.01191743,
       0.01300621, 0.01741337, 0.02912641])

In [30]:
features_names = preprocessor.get_feature_names_out()
features_names

array(['num__Trip_Distance_km', 'num__Passenger_Count', 'num__Base_Fare',
       'num__Per_Km_Rate', 'num__Per_Minute_Rate',
       'num__Trip_Duration_Minutes', 'cat__Time_of_Day_Afternoon',
       'cat__Time_of_Day_Evening', 'cat__Time_of_Day_Morning',
       'cat__Time_of_Day_Night', 'cat__Day_of_Week_Weekday',
       'cat__Day_of_Week_Weekend', 'cat__Traffic_Conditions_High',
       'cat__Traffic_Conditions_Low', 'cat__Traffic_Conditions_Medium',
       'cat__Weather_Clear', 'cat__Weather_Rain', 'cat__Weather_Snow'],
      dtype=object)

In [31]:
df_importances = pd.DataFrame({'Feature': features_names, 'Importance': importances_norm})

df_importances = df_importances.sort_values(by='Importance', ascending=True)

df_importances.head()

,Feature,Importance
1,num__Passenger_Count,0.006690
11,cat__Day_of_Week_Weekend,0.006701
10,cat__Day_of_Week_Weekday,0.006701
13,cat__Traffic_Conditions_Low,0.008144
2,num__Base_Fare,0.009161


In [32]:
fig = px.bar(df_importances, x='Importance',y='Feature',title='Importância das Features',
             orientation='h')

fig.update_xaxes(tickangle=False)
fig.show()

In [33]:
df_taxi_pricing.to_csv('./taxi_trip_pricing_cleaned.csv',index=False)

In [34]:
import joblib

joblib.dump(preprocessor, './preprocessor_regression.pkl')

['./preprocessor_regression.pkl']